In [1]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# get the list of all events from the beginning of the ufc
events_json_format = requests.get('http://ufc-data-api.ufc.com/api/v3/us/events').json()

In [3]:
# save the content of that content (json format) to a file
with open('../data/api/events.json', 'w') as json_file:
    json.dump(events_json_format, json_file)

In [4]:
# read the json file containing all the events since the first UFC event
df_events = pd.read_json('../data/api/events.json')

In [5]:
# make a simple request to get information about the API endpoint
url = 'http://ufc-data-api.ufc.com/api/v3/us/events/642991/fights'
response_test = requests.get(url)
print(response_test.status_code)
print(response_test.headers['Content-Type'])

200
application/json; charset=utf-8


In [6]:
def get_event_fights(event):
    """Make a get request to the UFC data API to get the fights of an event"""
    response = requests.get('http://ufc-data-api.ufc.com/api/v3/us/events/{}/fights'.format(event))
    return response.json()

In [ ]:
def get_all_fights(events):
    df_fights = pd.DataFrame()
    for event in events.itertuples():
        event_fights = json_normalize(get_event_fights(event.id))
        event_fights['event_date'] = event.event_date
        df_fights = df_fights.append(event_fights,sort=True)
    return df_fights

fights = get_all_fights(df_events)

In [ ]:
fights = fights.rename(index=str, 
                 columns={'fighter2_weight_class': 'weight_class',
                  'result.Method' : 'method',
                  'result.EndingTime' : 'ending_time',
                  'result.EndingRound' : 'ending_round',
                  'result.Submission' : 'submission',
                  'result.EndStrike' : 'end_strike',
                  'result.EndTarget' : 'end_target',
                  'result.EndPosition' : 'end_position',
                  'result.FightOfTheNight' : 'is_fotn'})

In [ ]:
fights.to_csv('../data/formated/fights.csv', 
          index=False, 
          columns=['fighter1_last_name',
                   'fighter2_last_name',
                   'weight_class', 
                   'fighter1reach', 
                   'fighter2reach', 
                   'fighter1height', 
                   'fighter2height',
                   'is_title_fight',
                   'is_main_event', 
                   'fighter1_rank', 
                   'fighter2_rank', 
                   'fighter1_is_winner', 
                   'fighter2_is_winner',
                   'method',
                   'ending_time',
                   'ending_round',
                   'submission',
                   'end_strike',
                   'end_target',
                   'end_position',
                   'is_fotn',
                   'event_date'
                  ])